In [1]:
import tatqa_utils
import pandas as pd

In [2]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [3]:
import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2

In [4]:
import os
with open('dataset_raw/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [5]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_community.cache import SQLiteCache

llm = ChatOpenAI(temperature=0)
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [39]:
def get_answer(llm, table, question):
    prompt = f"Given the following table, can you answer the following question? Please use format (value, scale) in your answer. Sclae can be million, thousand, precent, etc. Do not write explanation, just result.\nQuestion: {question} \n Table: {table}"
    res = llm.invoke(prompt)
    res2 = res.content
    print(res2)
    parts = res2.split(',')
    value = parts[0].replace('(','').replace('$','').strip()
    unit =  parts[1].replace(')','').replace('$','').strip()
    return (value, unit)
    
table = "[['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]"
get_answer(llm, table, 'What is the change in Fixed Price in 2019 from 2018?')



($306.2, million)


('306.2', 'million')

In [42]:
from tqdm.notebook import tqdm as log_progress

res = []
for i, item in log_progress(devdf.iterrows()):
    table = item['table']['table']
    for q in item['questions']:        
        if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from']: 
            r = get_answer(llm, table, q)
            
            if r is not None:
                if len(r) == 2:
                    (pred_value, pred_unit) = r
                elif len(r) == 1:
                    pred_value = r                
                else:
                    (pred_value, pred_unit) = (None, None)
            else:
                (pred_value, pred_unit) = (None, None)            
            #res.append({"table":table, "q":q, "pred":pred, "code": code})
            res.append(({"answer_type":q["answer_type"], "answer": q["answer"], 'scale': q["scale"]}, pred_value, pred_unit, q, "No Code"))


0it [00:00, ?it/s]

(-12.6, million)
(-22.22, percent)
(-94, million)
(-12.14, percent)
(2.1, percent)
(6.67, percent)
(-8.11, percent)
(172, million)
(40.5, million)
(121.5, million)
(26.82, percent)
(25.1, percent)
(2.98, percent)
(2.93, million)
(0.87, scale)
(-361, million)
(73, thousand)
(14.07, percent)
(4227.5, million)
(4,044, million)
(547.5, million)
(3728, thousand)
(16,458, thousand)
(154, million)
(104.05, percent)
(1.57, percent)
(1.9, percent)
(18, million)
(399.33, million)
(88.06, percent)
(0.2, million)
(40, percent)
(0.51, percent)
(0.7, scale: million)
(13,237.4, million)
(35796, million)
(64,509, thousand)
(62,740, thousand)
(14212, thousand)
(-67.6, percent)
(0, percent)
(3.0, percent)
(-9.9, percent)
(1.9, percent)
(2487.08, percent)
(70,056, thousand)
(67,210, thousand)
(-13.66, percent)
(-21.7, million)
(88.45, million)
(61.6, million)
(-78.06, percent)
(984, million)
(1,040, million)
(56, million)
(12,122, thousand)
(0.95, percent)
(0.05, million)
(38.23, percent)
(-11.92, percen

In [43]:
from tatqa_metric import TaTQAEmAndF1

metrics = TaTQAEmAndF1()

for ans, pred, pred_scale, _,_ in res:
    metrics(ans, pred, pred_scale)
pred_em, pred_f1, scale_score, op_score = metrics.get_overall_metric(reset=True)
print( pred_em, pred_f1, scale_score)

0.6951566951566952 0.6951566951566952 0.7108262108262108


In [44]:
import pickle
with open('res_p3_direct.pickle', 'wb') as f:
    pickle.dump(res,f)

In [24]:
res[0]


({'answer_type': 'arithmetic', 'answer': -12.6, 'scale': 'million'},
 None,
 None,
 {'uid': 'eb787966-fa02-401f-bfaf-ccabf3828b23',
  'order': 5,
  'question': 'What is the change in Other in 2019 from 2018?',
  'answer': -12.6,
  'derivation': '44.1-56.7',
  'answer_type': 'arithmetic',
  'answer_from': 'table-text',
  'rel_paragraphs': ['2'],
  'req_comparison': False,
  'scale': 'million'},
 'No Code')

In [20]:
len((1,2))

2